In [ ]:
"""
Analysis based on Hie et al., 2023 efficient evo paper Ext. Data Fig. 5:
Define a set of high fitness variants present in dataset (i.e. single mutations) found in at least 50% of AcrIIA4 variants ranked by EP
Define rankings of all single variants defined by ESM-based metric
Same approach but for DS (remember for this you are looking for LoF so ranking by negative likelihoods)
Across a set of sample sizes, i.e. log2, determine the number of high-fitness variants acquired using the ESM vs. DS approach
As a control, randomly select variants from the set of all possible single variants and also determine the number of high-fitness variants 
Plot strip plot to compare high-fitness identiifcation sensitivity

"""

In [ ]:
import pandas as pd
import numpy as np

# Load the DataFrame
data_path = '../ESMvsDS/2024June02_exp_vs_prediction_DMS_only.csv'  # Replace with the actual path to your CSV file
df = pd.read_csv(data_path)

# Convert necessary columns to numeric, coercing errors to NaN
df['avgprecision'] = pd.to_numeric(df['avgprecision'], errors='coerce')
df['totreads (all samples)'] = pd.to_numeric(df['totreads (all samples)'], errors='coerce')

# Drop rows with NaN values in the necessary columns
df = df.dropna(subset=['avgprecision', 'totreads (all samples)'])

# Filter by read count
read_threshold = 5000
df_filtered = df[df['totreads (all samples)'] >= read_threshold]

# Sort the DataFrame by avgprecision in descending order
df_sorted = df_filtered.sort_values(by='avgprecision', ascending=False)

# Define a percentile cutoff for the top-ranked sequences
percentile_cutoff = 95  # Example percentile cutoff, adjust as needed
cutoff_value = np.percentile(df_sorted['avgprecision'], percentile_cutoff)
top_sequences = df_sorted[df_sorted['avgprecision'] >= cutoff_value]
print(top_sequences)

# Compute the frequency of amino acids at each position
sequence_length = 11  # Given length of sequences in L5seq
amino_acid_positions = np.zeros((sequence_length, 20), dtype=int)
amino_acids = 'ACDEFGHIKLMNPQRSTVWY'
aa_to_index = {aa: i for i, aa in enumerate(amino_acids)}

for seq in top_sequences['L5seq']:
    if len(seq) == sequence_length:  # Only process sequences of the correct length
        for i, aa in enumerate(seq):
            if aa in aa_to_index:
                amino_acid_positions[i, aa_to_index[aa]] += 1
    else:
        print(f"Skipping sequence of incorrect length: {seq}")

# Create a DataFrame for amino acid frequencies
amino_acid_freq_df = pd.DataFrame(amino_acid_positions, columns=list(amino_acids))
amino_acid_freq_df.index = [f'Position {i+60}' for i in range(sequence_length)]  # Adjust positions from 60 to 70

# Display the frequencies
print(amino_acid_freq_df)

# Find the top three amino acids at each position
top_amino_acids = {}
for position in amino_acid_freq_df.index:
    top_3 = amino_acid_freq_df.loc[position].nlargest(3).index.tolist()
    top_amino_acids[position] = top_3

# Format the output as specified
formatted_output = []
for i, position in enumerate(range(60, 71)):
    formatted_output.extend([f"{position}{aa}" for aa in top_amino_acids[f"Position {position}"]])

print("Top N amino acids at each position:")
print(formatted_output)

# Optional: Plot the frequencies
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(14, 8))
sns.heatmap(amino_acid_freq_df.T, annot=True, cmap="YlGnBu")
plt.title('Amino Acid Frequency at Each Position')
plt.xlabel('Position in Sequence')
plt.ylabel('Amino Acid')
plt.show()



In [ ]:
import pandas as pd
import numpy as np

# Load the DataFrame
data_path = '../ESMvsDS/2024Feb_acrIIa4_hotspot_scores.csv'  # Replace with the actual path to your CSV file
df = pd.read_csv(data_path, index_col = 0)

# Verify the DataFrame structure
print(df.head())

# Ensure index positions are strings
df.index = df.index.astype(str)

# Verify the DataFrame structure
print("DataFrame head:")
print(df.head())

# Focus on positions 60-70
positions = [str(pos) for pos in range(60, 71)]  # Positions 60 to 70 as strings

# Check if positions exist in the DataFrame
existing_positions = df.index.intersection(positions)
if existing_positions.empty:
    print("None of the positions 60-70 are present in the DataFrame.")
else:
    print("Positions found in the DataFrame:", existing_positions)

# Create a pandas Series to store the rankings
rankings = pd.Series(dtype=float)

# Rank amino acids at each position and store in the Series
for position in existing_positions:
    sorted_values = df.loc[position].sort_values(ascending=False)
    for aa, score in sorted_values.items():
        rankings[f"{position}{aa}"] = score
        
# Sort the rankings in descending order
rankings = rankings.sort_values(ascending=False)

# Display the rankings
if rankings.empty:
    print("No rankings were created. Please check the DataFrame and positions.")
else:
    print("Rankings:")
    print(rankings)

In [ ]:
import pandas as pd

# Load the DataFrame
data_path = '../ESMvsDS/2024Feb_acrIIa4_hotspot_scores.csv'  # Replace with the actual path to your CSV file
df = pd.read_csv(data_path, index_col = 0)

# Ensure index positions are strings
df.index = df.index.astype(str)

# Verify the DataFrame structure
print("DataFrame head:")
print(df.head())


positions = [str(pos) for pos in range(1, 88)]  # all positions except the start
amino_acids = 'ACDEFGHIKLMNPQRSTVWY'

# Check if positions exist in the DataFrame
existing_positions = df.index.intersection(positions)
if existing_positions.empty:
    print("None of the positions 60-70 are present in the DataFrame.")
else:
    print("Positions found in the DataFrame:", existing_positions)

# Create a pandas Series to store the rankings
rankings = pd.Series(dtype=float)

# Rank amino acids at each position and store in the Series
for position in existing_positions:
    sorted_values = df.loc[position].sort_values(ascending=False)
    for aa, score in sorted_values.items():
        rankings[f"{position}{aa}"] = score

# Sort the rankings in descending order
rankings = rankings.sort_values(ascending=False)

# Display the rankings
if rankings.empty:
    print("No rankings were created. Please check the DataFrame and positions.")
else:
    print("Rankings:")
    print(rankings)

# Generate list of all possible single amino acid substitutions for positions 60-70
all_possible_substitutions = [f"{pos}{aa}" for pos in range(1, 88) for aa in amino_acids]

# Display all possible single amino acid substitutions
print("All possible single amino acid substitutions:")
print(all_possible_substitutions)


In [ ]:

def rank_ds_results(data_path):

    df = pd.read_csv(data_path, index_col = 0)

    # Ensure index positions are strings
    df.index = df.index.astype(str)

    # Verify the DataFrame structure
    print("DataFrame head:")
    print(df.head())


    positions = [str(pos) for pos in range(1, 88)]  # all positions except the start
    amino_acids = 'ACDEFGHIKLMNPQRSTVWY'

    # Check if positions exist in the DataFrame
    existing_positions = df.index.intersection(positions)
    if existing_positions.empty:
        print("None of the positions 60-70 are present in the DataFrame.")
    else:
        print("Positions found in the DataFrame:", existing_positions)

    # Create a pandas Series to store the rankings
    rankings = pd.Series(dtype=float)

    # Rank amino acids at each position and store in the Series
    for position in existing_positions:
        sorted_values = df.loc[position].sort_values(ascending=True) # here neg values result in LOF and may be hits
        for aa, score in sorted_values.items():
            rankings[f"{position}{aa}"] = score

    # Sort the rankings in descending order
    rankings = rankings.sort_values(ascending=True)

    # Display the rankings
    if rankings.empty:
        print("No rankings were created. Please check the DataFrame and positions.")
    else:
        print("Rankings:")
        print(rankings)
        
    return rankings
        
rank_ds_1 = rank_ds_results('../ESMvsDS/deepseq_analysis/2024June_acrIIa4_mut_matrix_T.csv') 

print(rank_ds_1)

In [ ]:
# Compute log2 sample sizes
#max_size = min(len(rankings), len(all_possible_substitutions), len(rank_ds_1))
max_size = 512
sample_sizes = [2 ** i for i in range(int(np.log2(max_size)) + 1)]

# Compute overlaps
overlap_results = []

for size in sample_sizes:
    
    #print(size)
    
    random_overlaps = []
    
    ranked_sample = set(rankings.head(size).index)
    
    overlap_esm_rankings = ranked_sample.intersection(set(formatted_output))
    overlap_esm_size = len(overlap_esm_rankings)
    
    ranked_ds = set(rank_ds_1.head(size).index)
    overlap_DS_rankings = ranked_ds.intersection(set(formatted_output))
    overlap_DS_size = len(overlap_DS_rankings)
    
    for _ in range(100):
        
        all_possible_sample = set(np.random.choice(all_possible_substitutions, size, replace=False))
        overlap_random_rankings = all_possible_sample.intersection(set(formatted_output))
        overlap_random_size = len(overlap_random_rankings)
        #print(overlap_random_size)
        random_overlaps.append(overlap_random_size)
    
    mean_overlap = np.mean(random_overlaps)
    std_overlap = np.std(random_overlaps)
    #print(std_overlap)
    #print(mean_overlap)
    
    overlap_results.append({
        'Sample Size': size,
        'Overlap Size ESM': overlap_esm_size,
        'Overlap Percentage ESM': overlap_esm_size / size * 100,
        'Overlap Size random mean': mean_overlap,
        'Std Overlap Size random': std_overlap,
        'Overlap Percentage random': overlap_random_size / size * 100, 
        'Overlap Size DS': overlap_DS_size,
        'Overlap Percentage size': overlap_DS_size / size * 100, 
    })

# Convert overlap results to DataFrame
overlap_df = pd.DataFrame(overlap_results)

# Display overlap results
print("Overlap Results:")
print(overlap_df)


# Optional: Plot the overlap results
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(overlap_df['Sample Size'], overlap_df['Overlap Size ESM'], marker='o', label='Overlap Size (ESM)')
#plt.plot(overlap_df['Sample Size'], overlap_df['Overlap Percentage ESM'], marker='x', label='Overlap Percentage (ESM)')
plt.errorbar(overlap_df['Sample Size'], overlap_df['Overlap Size random mean'], yerr=overlap_df['Std Overlap Size random'], fmt='o', capsize=10, elinewidth = 2, color = "orange", label='Random Overlap Size')
plt.plot(overlap_df['Sample Size'], overlap_df['Overlap Size random mean'], color = 'orange')
plt.plot(overlap_df['Sample Size'], overlap_df['Overlap Size DS'], marker='o', label='Overlap Size (DS)', color = "green")
plt.xscale('log', base=2)
plt.xlabel('Sample Size (log2 scale)')
plt.ylabel('Number of top-ranked AcrIIA4 variants within sample')
plt.title('Overlap between Ranked List and All Possible Substitutions')
plt.legend()
plt.grid(True)
plt.savefig("../ESMvsDS/2024June10_overlap_analysis_DS_vs_ESM.pdf", bbox_inches = "tight")
plt.show()

